# A small demo of optimizing nested functions

In [4]:
from opto.trace import node, bundle
from opto.trace.modules import apply_op
from opto.trace.modules import NodeContainer
import opto.trace.operators as ops


@bundle(description="[func_a] Returns a+1", trainable=True)
def func_a(a):
    return a + 1


@bundle(
    description="[func_b] Returns b+1", trainable=True, traceable_code=True
)  # traceable_code needs to be set True so we don't unpack the inputs and the function can see external dependencies
def func_b(b, fun):
    return fun(b) + 1


x = node(3)
y = func_b(x, func_a)


from opto.trace.optimizers import FunctionOptimizerV2

opt = FunctionOptimizerV2(func_a.parameters() + func_b.parameters())

i = 0
while True:

    y = func_b(x, func_a)
    if y != 10:
        feedback = "output should be 10."
    else:
        break
    opt.zero_feedback()
    opt.backward(y, visualize=False, feedback=feedback)
    opt.step(verbose="output")

    print(f"Iteration {i}")
    for p in opt.parameters:
        print(p.data)
    print("\n\n")
    i += 1

LLM response:
 {
"reasoning": "Based on the instruction, the issue seems to be that the output of the code, 'eval19', is currently 5 when it should be 10 according to the feedback. To achieve this, we need to adjust the logic inside functions __code6 and __code7. Currently, __code6 (func_a) increases its input by 1, and __code7 (func_b) applies func_a to its first argument and then increases the result by 1. For the output to be 10 with an input of 3 (int10=3), the operation must result in a triple increment or a different operation. One simple solution to reach an output of 10 from input 3 is to modify func_a to multiply its input by a factor that, when passed through func_b, results in the desired output. Since func_b applies func_a and then adds 1, func_a must transform 3 into a value that, when 1 is added to it, equals 10. Therefore, modifying func_a to multiply the input by 3 (thus, 3 * 3 = 9) and having func_b add 1 to the result (9 + 1 = 10) would achieve the desired outcome.",
